# Plots to do:
>- Loan amount per industry 
>- Influence of recession in loans 
>- Correlation matrix
>- Influence of long terms loans (Term) in terms of loan approval
>- Map with rate per state
>- Influence of loan size

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objects as go

In [13]:
pd.set_option('display.max_rows', 100)
df_x_copy = pd.read_csv("/Users/macbookpro/Documents/Cours/Singapour/S2/Data discovery and data mining/Project/Project/Xtrain.csv",index_col='Id')
df_y_copy = pd.read_csv("/Users/macbookpro/Documents/Cours/Singapour/S2/Data discovery and data mining/Project/Project/Ytrain.csv",index_col='Id')

# Load the SBA loan data and make a copy for exploration

df = pd.concat([df_x_copy,df_y_copy],join='inner',axis=1)
df_used = df.copy(deep=True)
df_used

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,ChargeOff
Id,,,,,,,,,,,,,,,,,,,,,
0,"NPE Realty, LLC",Pembroke Pines,FL,33027,"TD BANK, NATIONAL ASSOCIATION",DE,334516,16-Apr-10,2010,130,...,0,1,N,N,1-May-10,"$2,000,000.00",$0.00,"$2,000,000.00","$1,500,000.00",0
1,"KEVCO CONSTRUCTION, LLC",SANDY,UT,84094,ZIONS FIRST NATIONAL BANK,UT,236115,30-Jan-10,2010,33,...,0,1,N,N,1-Feb-10,"$17,000.00",$0.00,"$17,000.00","$17,000.00",1
2,EAST L A SHOE REPAIR,LOS ANGELES,CA,90022,BANK OF AMERICA NATL ASSOC,NC,811430,29-Jul-02,2002,60,...,1,1,Y,N,31-Aug-02,"$17,000.00",$0.00,"$17,000.00","$8,500.00",0
3,"RUSSLER, DANIEL C, MD",LODI,WI,53555,ASSOCIATED BANK NATL ASSOC,WI,0,10-Jun-94,1994,160,...,1,0,N,N,31-Jul-94,"$137,000.00",$0.00,"$137,000.00","$123,300.00",0
4,AFFORDABLE FAMILY DENISTRY LLC,LEBANON,MO,65536,COMMERCE BANK,MO,621210,6-Feb-06,2006,101,...,1,1,T,N,31-Mar-06,"$46,665.00",$0.00,"$25,000.00","$12,500.00",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,WESTTOWN GULF,WEST CHESTER,PA,19382,CITIZENS BANK NATL ASSOC,RI,445120,29-Aug-03,2003,84,...,1,1,Y,N,30-Sep-03,"$46,231.00",$0.00,"$30,000.00","$15,000.00",0
49996,HS ELECTRIC,ENCINO,CA,91316,BANK OF AMERICA NATL ASSOC,NC,541513,12-Jul-04,2004,77,...,1,1,N,N,31-Jul-04,"$49,750.00",$0.00,"$50,000.00","$25,000.00",1
49997,"HARDIN GRAIN ELEVATOR, INC.",HARDIN,KY,42048,COMMUNITY FINAN SERVICES BK,KY,0,4-Oct-91,1992,223,...,1,0,N,N,31-Jan-92,"$260,000.00",$0.00,"$260,000.00","$221,000.00",0


In [54]:
import requests
req = requests.get('https://raw.githubusercontent.com/plotly/datasets/master/1962_2006_walmart_store_openings.csv')
url_content = req.content
csv_file = open('/Users/macbookpro/Documents/Cours/Singapour/S2/Data discovery and data mining/Project/US_states_per_year.csv', 'wb')

csv_file.write(url_content)
csv_file.close()


In [204]:
import plotly.graph_objects as go

x = list(df_used_test['DisFY'].unique())
y0,y1 = [],[]
x.sort()
for x_ in (x):
    df_target = df_used_test[df_used_test.DisFY == x_]
    y0.append(df_target[df_target['ChargeOff'] == 0].shape[0])
    y1.append(df_target[df_target['ChargeOff'] == 1].shape[0])
layout = go.Layout(title_text="Percentage charged off loans in function of disbursment FY")
fig = go.Figure(layout=layout)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=1
),paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')


fig.update_layout(title_text="Number of charged off loans in function of disbursment FY", title_x=0.5)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.add_trace(go.Scatter(x=x, y=y1, fill='tonexty',stackgroup='one',name="% charged off loans")) # fill to trace0 y
fig.add_trace(go.Scatter(x=x, y=y0, fill='tonexty',stackgroup='one',name="% not charged off loans")) # fill down to xaxis
fig.show()

In [205]:
import plotly.graph_objects as go

x = list(df_used_test['DisFY'].unique())
y0,y1 = [],[]
x.sort()
for x_ in (x):
    df_target = df_used_test[df_used_test.DisFY == x_]
    y0.append(df_target[df_target['ChargeOff'] == 0].shape[0]/df_target.shape[0])
    y1.append(df_target[df_target['ChargeOff'] == 1].shape[0]/df_target.shape[0])

fig = go.Figure()
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=1
),paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(title_text="Percentage of charged off loans in function of years", title_x=0.5)
fig.add_trace(go.Scatter(x=x, y=y1, fill='tonexty',name="% charged off loans")) # fill to trace0 y
fig.add_trace(go.Scatter(x=x, y=y0, fill='tonexty',name="% not charged off loans")) # fill down to xaxis
fig.show()

In [57]:
df_states= pd.read_csv('/Users/macbookpro/Documents/Cours/Singapour/S2/Data discovery and data mining/Project/US_states.csv')

dict_sum = dict(df_used.groupby('State').sum()['ChargeOff'])
dict_ = dict(df_used['State'].value_counts())
dict_res = {}
for k in dict_.keys():
    dict_res.update({k:round(float(dict_sum[k]/dict_[k]),3)})

In [168]:

fig = go.Figure(data=go.Choropleth(
    locations=df_states['code'], # Spatial coordinates
    z = list_res, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Percentage of charged off loans",
))

fig.update_layout(autosize=True,width=1500, height=800,
title_x=0.5,
    title_text = 'Percentage of charged off loans per state in function of state per 1970-2013',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [94]:
df_used[['ApprovalDate', 'DisbursementDate']] = df_used[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)
df_used['DisFY'] = df_used['DisbursementDate'].map(lambda x: x.year).apply('int64')
df_used_test = df_used[df_used['DisFY'].isin([i for i in range(1970,2014)])]

array([2010, 2002, 1994, 2006, 2008, 1999, 2007, 2004, 2001, 2011, 1996,
       1984, 1989, 2000, 2003, 2009, 1997, 1992, 1990, 2005, 1993, 1985,
       1983, 1995, 1991, 1998, 1987, 2012, 1982, 1988, 1986, 2013, 1981,
       1978, 1979, 1980, 1977, 1973, 1975, 1976, 1972, 1974, 1970])

In [150]:
df_states= pd.read_csv('/Users/macbookpro/Documents/Cours/Singapour/S2/Data discovery and data mining/Project/US_states.csv')
def find_data(start,end):
    df_target = df_used_test[df_used_test['DisFY'].isin([i for i in range(start,end)])]
    dict_sum = dict(df_target.groupby('State').sum()['ChargeOff'])
    dict_ = dict(df_target['State'].value_counts())
    dict_res = {}
    for k in dict_.keys():
        dict_res.update({k:round(float(dict_sum[k]/dict_[k]),3)})
    return dict_res,df_target


In [167]:

import plotly.graph_objs as go
from plotly import tools
years = [1980,1990,2000,2007,2010,2014]
annotations_ = []
#fig = tools.make_subplots(rows=1, cols=len(years) ,print_grid=False);
data = []
text_ = []
layout = dict(title = 'Evolution of charged off loans percentage in function of state and disbursment FY',
    title_x=0.1,
    showlegend = True,
    autosize = False,
    width = 1000,
    height = 900,
    hovermode = False,
    legend = dict(
        x=0.7,
        y=-0.1,
        bgcolor="rgba(255, 255, 255, 0)",
        font = dict( size=1 ), ))
for i in range(len(years)-1):
    geo_key = 'geo'+str(i+1) if i != 0 else 'geo'
    dict_res,df = find_data(years[i],years[i+1]-1)
    list_res = []
    for code in list(df_states['code']):
        list_res.append(dict_res[code])
    
    text_.append('Nb of loans: {} <br> Overall % charged off {}'.format(df.shape[0],round(df['ChargeOff'].sum()/df.shape[0],3)))
    if i == 0:

        data.append(
        dict(
            type = 'choropleth',
            locations=df_states['code'], # Spatial coordinates
            z = list_res, # Data to be color-coded
            locationmode = 'USA-states', # set of locations match entries in `locations`
            colorscale = 'Reds',
            showscale=True,
            colorbar_title = "Percentage of Charge off loans",
            geo = geo_key,
            
            ))
        
    else:
        data.append(
        dict(
            type = 'choropleth',
            locations=df_states['code'], # Spatial coordinates
            z = list_res, # Data to be color-coded
            locationmode = 'USA-states', # set of locations match entries in `locations`
            colorscale = 'Reds',
            showscale=False,
            colorbar_title = "Percentage of Charge off loans",
            geo = geo_key,
            
            ))
    ''' data =go.Choropleth(
        locations=df_states['code'], # Spatial coordinates
        z = list_res, # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'Reds',
        colorbar_title = "Percentage of charged off loans",
    )'''
    #fig.append_trace(data, 1, i+1)
    data.append(
        dict(
            type = 'scattergeo',
            showlegend = False,
            lon = [-100],
            lat = [51],
            geo = geo_key,
            text = ['{}-{}'.format(years[i],years[i+1]-1)],
            mode = 'text',
        )
    )
    layout[geo_key] = dict(
       
        scope = 'usa',
        showland = True,
        landcolor = 'rgb(229, 229, 229)',
        showcountries = False,
        domain = dict( x = [], y = [] ),
        subunitcolor = "rgb(255, 255, 255)",
    )


z = 0
COLS = 2
ROWS = 3
i_cur = -1
for y in reversed(range(ROWS)):
    for x in range(COLS):
        i_cur += 1
        try:
            geo_key = 'geo'+str(z+1) if z != 0 else 'geo'
            layout[geo_key]['domain']['x'] = [float(x)/float(COLS), float(x+1)/float(COLS)]
            layout[geo_key]['domain']['y'] = [float(y)/float(ROWS), float(y+1)/float(ROWS)]
            annotations_.append(dict(
        x=0.05+0.85*(x),
        y=0.03+0.35*y,
        xref='paper',
        yref='paper',
        text=text_[i_cur],
        showarrow = False))
            z=z+1
            if z > 42:
                break

        except:pass

fig = go.Figure(data=data, layout=layout)
fig.update_layout(width=800,annotations = annotations_)
fig.show()


In [7]:
df_used_test = df_used.copy(deep=True)
print(df_used_test.shape[0])
df_used_test[['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']] = df_used_test[['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']].astype('string')
df_used_test[['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']] = df_used_test[['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', '').replace('.','')).astype('int64')
print(df_used_test.shape[0])
df_used_test['ApprovalFY'] = df_used_test['ApprovalFY'].astype('string')
df_used_test['ApprovalFY'] = df_used_test['ApprovalFY'].apply(lambda x: x.replace('A', '')).astype('int64')

df_used_test['Industry'] = df_used_test['NAICS'].astype('str').apply(lambda x: x[:2])

"""NAICS code: https://www.census.gov/cgi-bin/sssd/naics/naicsrch?chart=2017"""

df_used_test['Industry'] = df_used_test['Industry'].map({
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale trade',
    '44': 'Retail trade',
    '45': 'Retail trade',
    '48': 'Transportation and Warehousing',
    '49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
})
print( df_used_test.shape[0])
df_used_test['NewExist'] = df_used_test['NewExist'].astype('int64')
df_used_test = df_used_test[df_used_test['NewExist'].isin([1,2])]
print('NewExist',df_used_test.shape[0])

df_used_test['IsFranchised'] = df_used_test.FranchiseCode
df_used_test[df_used_test['IsFranchised'] <= 1].IsFranchised = 0 
df_used_test[df_used_test['IsFranchised'] > 1].IsFranchised = 1
print('IsFranchised',df_used_test.shape[0])

df_used_test = df_used_test[df_used_test['UrbanRural'].isin([0,1,2])]
print(df_used_test.shape[0])
df_used_test = df_used_test[df_used_test['LowDoc'].isin(['Y','N'])]
print(df_used_test.shape[0])
#df_used = df_used[df_used['RevLineCr'].isin(['Y','N','0'])]
#print(df_used.shape[0])

df_used_test['LowDoc'] = df_used_test['LowDoc'].map({'Y': '1','N': '0'}).astype('int64')
#df_used['RevLineCr'] = df_used['RevLineCr'].map({'Y': '1','N': '0','0':2}).astype('int64')
df_used_test['ApprovalFY'] = df_used_test['ApprovalFY'].astype('string')
df_used_test['ApprovalFY'] = df_used_test['ApprovalFY'].apply(lambda x: x.replace('A', '')).astype('int64')
df_used_test[['ApprovalDate', 'DisbursementDate']] = df_used_test[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)
df_used_test['DisFY'] = df_used_test['DisbursementDate'].map(lambda x: x.year).apply('int64')
df_used_test['ratioApproved'] = df_used_test['SBA_Appv'] / df_used_test['GrAppv']
df_used_test['ratioBankAmount'] = df_used_test['GrAppv'] / df_used_test['DisbursementGross']
df_used_test.loc[df_used_test['GrAppv'] > df_used_test['DisbursementGross'],'BalanceComp'] = 1
df_used_test.loc[df_used_test['GrAppv'] < df_used_test['DisbursementGross'],'BalanceComp'] = 2
df_used_test.loc[df_used_test['GrAppv'] == df_used_test['DisbursementGross'],'BalanceComp'] = 0
df_used_test.loc[df_used_test['BankState'] == df_used_test['State'],'FlagState'] = 1
df_used_test.loc[df_used_test['BankState'] != df_used_test['State'],'FlagState'] = 0
df_used_test['CreateJobPer'] = df_used_test['CreateJob'] / df_used_test['NoEmp']
df_used_test['RetainedJobPer'] = df_used_test['RetainedJob'] / df_used_test['NoEmp']
df_used_test['BalanceComp'].astype('int64')
print(df_used_test.shape[0])


100000


KeyError: &quot;None of [Index([&#39;DisbursementGross&#39;, &#39;BalanceGross&#39;, &#39;GrAppv&#39;, &#39;SBA_Appv&#39;], dtype=&#39;object&#39;)] are in the [columns]&quot;

In [211]:

df_used_test['Industry'] = df_used['NAICS'].astype('str').apply(lambda x: x[:2])

"""NAICS code: https://www.census.gov/cgi-bin/sssd/naics/naicsrch?chart=2017"""

df_used_test['Industry'] = df_used_test['Industry'].map({
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale trade',
    '44': 'Retail trade',
    '45': 'Retail trade',
    '48': 'Transportation and Warehousing',
    '49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
})
df_used_test = df_used_test.dropna(subset=['Industry'])

In [244]:
df_used_test[df_used_test['Industry'].isin(['Management of Companies and Enterprises'])]

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,ChargeOff,DisFY,Industry
Id,,,,,,,,,,,,,,,,,,,,,
178,RR&R Dynamic Business Solution,SOUTH OZONE PARK,NY,11420,HSBC BK USA NATL ASSOC,VA,551112,2005-07-27,2005,59,...,N,N,2005-08-31,"$100,000.00",$0.00,"$100,000.00","$50,000.00",1,2005,Management of Companies and Enterprises
584,COLUMBIA BUSINESS INSIGHT,VANCOUVER,WA,98683,U.S. BANK NATIONAL ASSOCIATION,OH,551114,2005-11-07,2006,84,...,Y,N,2006-05-31,"$213,792.00",$0.00,"$50,000.00","$25,000.00",0,2006,Management of Companies and Enterprises
3739,SOLLERS PARTNERSHIP,MORRIS,MN,56267,BANK OF THE WEST,MN,551112,1997-06-23,1997,120,...,N,Y,1997-11-30,"$100,000.00",$0.00,"$100,000.00","$80,000.00",0,1997,Management of Companies and Enterprises
7716,WS Live LLC,PLANO,TX,75093,JPMORGAN CHASE BANK NATL ASSOC,IL,551112,2006-09-20,2006,120,...,N,N,2006-12-31,"$2,000,000.00",$0.00,"$2,000,000.00","$1,500,000.00",0,2006,Management of Companies and Enterprises
10561,BEAR TRACK L L C,WILLIAMSBURG,MI,49690,"FIRSTMERIT BANK, N.A.",MI,551112,1998-08-17,1998,249,...,0,N,1998-09-30,"$850,000.00",$0.00,"$850,000.00","$637,500.00",0,1998,Management of Companies and Enterprises
17614,CLAY PROPERTIES LLC,GREAT FALLS,MT,59405,MOUNTAIN W. BANK NATL ASSOC,MT,551112,2000-12-26,2001,240,...,0,N,2002-01-31,"$628,000.00",$0.00,"$628,000.00","$471,000.00",0,2002,Management of Companies and Enterprises
19276,SHERI L. WILKERSON DBA NASH NU,NASH,TX,75569,CAPITAL ONE NATL ASSOC,VA,551112,2004-11-23,2005,29,...,N,N,2004-12-31,"$35,000.00",$0.00,"$35,000.00","$17,500.00",1,2004,Management of Companies and Enterprises
19996,AURORA LEASING LLC,TUCSON,AZ,85706,UMPQUA BANK,OR,551112,1999-07-09,1999,180,...,0,N,1999-07-31,"$252,000.00",$0.00,"$252,000.00","$189,000.00",0,1999,Management of Companies and Enterprises
22228,Restaurant Entertainment Ventu,INDIANAPOLIS,IN,46202,JPMORGAN CHASE BANK NATL ASSOC,IL,551112,2005-02-23,2005,55,...,N,N,2005-02-28,"$180,000.00",$0.00,"$180,000.00","$90,000.00",1,2005,Management of Companies and Enterprises


In [369]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
y0,y1 = [],[]
x = list(df_used_test['Industry'].unique())
for x_ in x:
    df_target = df_used_test[df_used_test['Industry'].isin([x_])]
    y0.append(df_target[df_target.ChargeOff == 0].shape[0])
    y1.append(df_target[df_target.ChargeOff == 1].shape[0])


fig = go.Figure(data=[
    go.Bar(name='% charged off loans', x=x, y=y1,text=y1,marker_color='rgb(55, 83, 109)'),
    go.Bar(name='% not charged off loans', x=x, y=y0,text=y0,marker_color='indianred')
])
fig.update_layout(barmode='stack')
fig.update_layout(
    title='Percentage of charged off loans in function of industry sectors',
    title_x=0.5,
    yaxis=dict(
        showgrid=False,
        showline=False,
    
    ),
    
    xaxis=dict(
        zeroline=False,
        showline=False,
        showgrid=False,
       
    ),
    autosize=False,

    width=1500,
    height=1000,
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(margin={"r":0,"t":100,"l":0,"b":300})
fig.update_layout(margin={"r":0,"t":100,"l":0,"b":300})

fig.show()

In [303]:
#f_used_test[df_used_test['DisFY']  + 
df_used_test['Offset'] = df_used_test['Term'].apply(lambda x: pd.DateOffset(months=x))

In [311]:
df_used_test['Offset'] = df_used_test['Term'].apply(lambda x: pd.DateOffset(months=x))
df_used_test['Offset_end'] = df_used_test['DisbursementDate'] +df_used_test['Offset']

In [328]:
from datetime import datetime
df_used_test['Offset'] = df_used_test['Term'].apply(lambda x: pd.DateOffset(months=x))
df_used_test['Offset_end'] = df_used_test['DisbursementDate'] +df_used_test['Offset']
df_used_test['IsActiveRecession'] = 0
df_used_test.loc[(df_used_test['Offset_end'] >= datetime(year=2007, month=12, day=1)) & (df_used_test['Offset_end'] <= datetime(year=2010, month=12, day=1)),'IsActiveRecession']= 1

In [346]:
df_used_test[df_used_test['IsActiveRecession'] ==1].groupby('Industry').sum()

,Zip,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,ChargeOff,DisFY,IsActiveRecession
Industry,,,,,,,,,,,,
Accommodation and Food Services,44850115,626104255,49271,8522,1213.0,2459,6055,7753833,998,592,1737710,867
Administrative and Support and Waste Management and Remediation Services,29147601,326903419,29151,3464,727.0,1079,3202,1362583,645,425,1166832,582
"Agriculture, Forestry, Fishing and Hunting",4017929,8151788,7738,417,101.0,71,260,66,57,23,144014,72
"Arts, Entertainment, and Recreation",11063115,140503381,11461,1221,271.0,413,873,640663,220,134,394798,197
Construction,55214552,245171420,48824,6885,1220.0,1639,5510,48571,1170,790,2073237,1034
Educational Services,5478236,61771491,5078,2129,130.0,1287,849,442386,110,74,202483,101
Finance and Insurance,9964711,87943229,8163,853,194.0,228,843,170151,184,134,336831,168
Health Care and Social Assistance,28041647,319122543,34875,3341,676.0,807,2231,246757,516,253,1027710,513
Information,11420316,105406502,10568,1100,252.0,334,872,143,224,151,410968,205


In [347]:
df_used_test[df_used_test['IsActiveRecession'] ==0].groupby('Industry').sum()

,Zip,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,ChargeOff,DisFY,IsActiveRecession
Industry,,,,,,,,,,,,
Accommodation and Food Services,172262826,2302649048,349842,34488,4634.0,15849,22898,35061026,3624,1661,6393898,0
Administrative and Support and Waste Management and Remediation Services,73622556,800423915,100500,13271,1811.0,2623,9020,4588324,1506,779,2856690,0
"Agriculture, Forestry, Fishing and Hunting",19839028,42400240,40761,1838,496.0,293,923,288,379,110,750394,0
"Arts, Entertainment, and Recreation",32517785,455835559,62405,5020,879.0,1526,6754,1592619,590,325,1280077,0
Construction,169442777,724910626,230444,30450,3518.0,5610,17333,1070642,3038,1595,6134495,0
Educational Services,16966427,203046628,26275,2347,443.0,564,1902,1758961,361,197,665886,0
Finance and Insurance,29813564,258613706,40710,3405,593.0,797,2660,92940,545,304,990707,0
Health Care and Social Assistance,116613693,1289537448,241516,17356,2787.0,4191,8779,1779650,1705,664,4151535,0
Information,30400472,275846794,39614,4344,694.0,1456,2104,288395,512,297,1076273,0


In [364]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
y0,y1,y00,y11 = [],[],[],[]
x = list(df_used_test['Industry'].unique())
for x_ in x:
    df_target = df_used_test[df_used_test['Industry'].isin([x_])]
    df_target  =df_target[df_target['IsActiveRecession'] ==0 ]
    y00.append(df_target[df_target.ChargeOff == 0].shape[0]/df_target.shape[0])
    y11.append(df_target[df_target.ChargeOff == 1].shape[0]/df_target.shape[0])

    df_target = df_used_test[df_used_test['Industry'].isin([x_])]
    df_target  =df_target[df_target['IsActiveRecession'] == 1 ] 
    y0.append(df_target[df_target.ChargeOff == 0].shape[0]/df_target.shape[0])
    y1.append(df_target[df_target.ChargeOff == 1].shape[0]/df_target.shape[0])

fig = go.Figure(data=[
    go.Bar(name='% charged off loans during Great Recession', x=x, y=y1,text=y1,offsetgroup=0,marker_color='rgb(55, 83, 109)'),
    go.Bar(name='% not charged off loans during Great Recession', x=x, y=y0,text=y0,offsetgroup=0,base=y1,marker_color='indianred'),
    go.Bar(name='% charged off loans otherwise', x=x, y=y11,text=y11,offsetgroup=1,marker_color='rgb(55, 83, 109)',opacity=.7),
    go.Bar(name='% not charged off loans otherwise', x=x, y=y00,text=y00,offsetgroup=1,base=y11,marker_color='indianred',opacity=.7),
])
#fig.update_layout(barmode='stack')
fig.update_layout(
    title='Percentage of charged off loans in function of industry sectors and Great Recession',
    title_x=0.5,
    yaxis=dict(
        showgrid=False,
        showline=False,
    
    ),
    
    xaxis=dict(
        zeroline=False,
        showline=False,
        showgrid=False,
       
    ),
    autosize=False,
    width=1500,
    height=1000,
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    bargap=0.15,
    bargroupgap=0.15,
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='inside')
fig.update_layout(margin={"r":0,"t":100,"l":0,"b":300})

fig.show()

In [371]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
y0,y1 = [],[]
x = list(df_used_test['Industry'].unique())
for x_ in x:
    df_target = df_used_test[df_used_test['Industry'].isin([x_])]
    y0.append(df_target[df_target.ChargeOff == 0].shape[0]/df_target.shape[0])
    y1.append(df_target[df_target.ChargeOff == 1].shape[0]/df_target.shape[0])


fig = go.Figure(data=[
    go.Bar(name='% charged off loans', x=x, y=y1,text=y1,marker_color='rgb(55, 83, 109)'),
    go.Bar(name='% not charged off loans', x=x, y=y0,text=y0,marker_color='indianred')
])
fig.update_layout(barmode='stack')
fig.update_layout(
    title='Percentage of charged off loans in function of industry sectors',
    title_x=0.5,
    yaxis=dict(
        showgrid=False,
        showline=False,
    
    ),
    
    xaxis=dict(
        zeroline=False,
        showline=False,
        showgrid=False,
       
    ),
    autosize=False,

    width=1500,
    height=1000,
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(margin={"r":0,"t":100,"l":0,"b":300})
fig.update_layout(margin={"r":0,"t":100,"l":0,"b":300})

fig.show()